In [9]:
import os
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString

import numpy as np
import pandas as pd

import tifffile

# pack multiple images into one ome.tiff file

In [ ]:
path_ometiff = "path/to/save/ome.tiff"
im_paths = ["path/to/image_1", "path/to/image_2", "path/to/image_3"]
im_names = ["name_1", "name_2", "name_3"]

im_arrays = [tifffile.imread(path) for path in im_paths]
im_arrays_stack = np.stack(im_arrays, axis=0).astype(np.uint16)
tifffile.imwrite(
    path_ometiff,
    im_arrays_stack,
    metadata={"axes": "CYX", "Channel": {"Name": im_names}},
    ome=True,
)

# get metadata from ome.tiff

In [5]:
# Load the OME-TIFF file
file_path = "/mnt/nfs/home/wenruiwu/projects/bidmc-jiang-rcc/output/data/03_ometiff/TMA543_run1=reg001_run2=reg021.ome.tiff"
im = tifffile.TiffFile(file_path)
im

<tifffile.TiffFile 'TMA543_run1=reg…=reg021.ome.tiff'>

In [7]:
im_array = im.asarray()
im_array.shape

(46, 5472, 5952)

In [ ]:
# Raw XML string extracted from the OME-TIFF metadata
xml_str = im.ome_metadata

# Format the XML string into a more readable format with indentation
formatted_xml = parseString(xml_str).toprettyxml(indent="    ")
print(formatted_xml)


<?xml version="1.0" ?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:99402df0-a70b-11ef-893a-0154ee850d17" Creator="tifffile.py 2024.2.12">
    <Image ID="Image:0" Name="Image0">
        <Pixels ID="Pixels:0" DimensionOrder="XYCZT" Type="uint16" SizeX="5952" SizeY="5472" SizeC="46" SizeZ="1" SizeT="1">
            <Channel ID="Channel:0:0" SamplesPerPixel="1" Name="DAPI">
                <LightPath/>
            </Channel>
            <Channel ID="Channel:0:1" SamplesPerPixel="1" Name="CD45">
                <LightPath/>
            </Channel>
            <Channel ID="Channel:0:2" SamplesPerPixel="1" Name="CD3e">
                <LightPath/>
            </Channel>
            <Channel ID="Channel:0:3" SamplesPerPixel="1" Name="CD8">
                <LightPath/>
            <

In [11]:
# Parse the XML string into an ElementTree object
root = ET.fromstring(xml_str)

# Define the namespace for the OME XML schema
# The namespace is required to correctly locate elements in the XML structure
namespace = {"ome": "http://www.openmicroscopy.org/Schemas/OME/2016-06"}

# Iterate through all "Channel" elements in the XML
# The ".//ome:Channel" syntax finds all "Channel" elements at any level of the XML
channels = []
for channel in root.findall(".//ome:Channel", namespace):
    channel_id = channel.get("ID")
    name = channel.get("Name")
    samples_per_pixel = channel.get("SamplesPerPixel")
    channels.append(
        {"ID": channel_id, "Name": name, "SamplesPerPixel": samples_per_pixel}
    )

df = pd.DataFrame(channels)
print(df)


              ID         Name SamplesPerPixel
0    Channel:0:0         DAPI               1
1    Channel:0:1         CD45               1
2    Channel:0:2         CD3e               1
3    Channel:0:3          CD8               1
4    Channel:0:4          CD4               1
5    Channel:0:5       CD45RO               1
6    Channel:0:6       CD45RA               1
7    Channel:0:7         CD69               1
8    Channel:0:8         CD57               1
9    Channel:0:9         CD56               1
10  Channel:0:10        FoxP3               1
11  Channel:0:11         CD28               1
12  Channel:0:12         CD86               1
13  Channel:0:13        T-bet               1
14  Channel:0:14       TCF1_7               1
15  Channel:0:15        IFN-y               1
16  Channel:0:16    GranzymeB               1
17  Channel:0:17     Tox_Tox2               1
18  Channel:0:18        Tim-3               1
19  Channel:0:19         PD-1               1
20  Channel:0:20        LAG-3     